In [6]:
from tensorflow.keras.preprocessing.image import(
    ImageDataGenerator,
    img_to_array,
    load_img
)
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import(
    AveragePooling2D,
    Dropout,
    Flatten,
    Dense,
    Input
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.utils import to_categorical

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
import numpy as np
import os

import cv2

In [9]:
LR = 1e-4 
EPOCHS = 20
BS = 32

# Put the Dataset Path Here
DATA_PATH = "./dataset"
Class = ["with_mask", "without_mask"]

In [10]:
#IMAGE & LABEL Gathering
imgs = []
labels = []

for cl in Class:
    path = os.path.join(DATA_PATH, cl)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        image = load_img(img_path, target_size=(224, 224))
        image = img_to_array(image)
        image = preprocess_input(image)

        imgs.append(image)
        labels.append(cl)
print(len(labels))
print(imgs[0][0:2])

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x0000024FFF57AF90>

In [ ]:
# Label encoding and spliting the dataset into training and testing.
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

data = np.array(imgs)
labels = np.array(labels)

(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.20, stratify=labels, random_state=42)

In [ ]:
# Image Augmentation for increasing dataset count.
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

In [ ]:
# Using MobileNetV2 as Base Network for this model with 224 x 224 x 3 as input shape.
baseModel = MobileNetV2(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

In [ ]:
# Appending the own custom Head Network with 1 Average Pooling Layer, Flatten, 2 Dense layers with 0.5 Dropout, Relu Activation for Dense Layers and Softmax Activation for Output Layer.
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7,7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)

for layer in baseModel.layers:
    layer.trainable = False
model.summary()

In [ ]:
# Creating Optimizer of Adam with 1e-4 LR and complie the model after that.
opt = Adam(lr=LR, decay=LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

In [ ]:
# Train a model with train partition for training and test partition for validation
H = model.fit(
    aug.flow(trainX, trainY, batch_size=BS),
    steps_per_epoch = len(trainX) // BS,
    validation_data=(testX, testY),
    validation_steps= len(testX) // BS,
    epochs = EPOCHS
)

In [ ]:
#model.load_weights("mask_detector.model")

In [ ]:
#Predicting the Whole Test Partition for performance measure. 
predIdxs = model.predict(testX, batch_size=BS)
predIdxs = np.argmax(predIdxs, axis=1)

In [ ]:
#Ploting a confusion metric with Precision and Recall value.
print(classification_report(testY.argmax(axis=1), predIdxs, target_names = lb.classes_))

In [ ]:
#Saving a Model.
model.save("mask_detector.model", save_format="h5")

In [ ]:
# Plotting and saving the Accuracy graph of training and validation. 
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
#plt.savefig("train_acc.png")

In [ ]:
#Plotting and saving the Loss graph of training and validation. 
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.title("Training Loss")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
#plt.savefig("train_loss.png")

In [ ]:
# predicting custom image
try_img = load_img("try\\me_without.jpg", target_size=(224,224))
try_img = img_to_array(try_img)
image = preprocess_input(try_img)
pred = model.predict(image[np.newaxis, :, : ,:])
print(pred,Class[np.argmax(pred)])